In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import gradio as gr


In [18]:
# Mount Google Drive if the dataset is stored there
from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/fraud_detection/fraud_dataset_updated.csv')  # Adjust path if needed
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,from_acc,to_acc,amount,sender_balance,receiver_balance,time_of_day,is_fraud,third_party_involved,sender,receiver
0,ACC1010,ACC1005,2246.47,5267.85,8669.46,12.0,0,0,NaN,NaN
1,ACC1005,ACC1005,423.83,10520.22,13767.07,2.0,0,0,NaN,NaN
2,ACC1004,ACC1002,7698.64,7642.43,16151.18,14.0,1,0,NaN,NaN
3,ACC1009,ACC1010,9029.06,2390.50,2591.14,11.0,1,0,NaN,NaN
4,ACC1005,ACC1009,2756.46,9888.83,9566.68,11.0,0,0,NaN,NaN


In [21]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the model
joblib.dump(model, 'fraud_model.pkl')


['fraud_model.pkl']

In [22]:
# Predict and evaluate
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[161   0]
 [  8  91]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.98       161
           1       1.00      0.92      0.96        99

    accuracy                           0.97       260
   macro avg       0.98      0.96      0.97       260
weighted avg       0.97      0.97      0.97       260



In [23]:
def is_micro_transaction(amount, threshold=5.0):
    """
    Checks if a transaction is a micro transaction.

    Args:
        amount (float): Transaction amount.
        threshold (float): Threshold below which a transaction is considered 'micro'.

    Returns:
        bool: True if micro transaction, else False.
    """
    return amount < threshold


In [24]:
# Global list to store history
history = []


In [25]:
def predict_fraud(type_, amount, oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest):
    # Load model
    model = joblib.load('fraud_model.pkl')

    # Format the input
    input_data = [[type_, amount, oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest]]

    # Prediction
    prediction = model.predict(input_data)[0]
    result = "Fraudulent" if prediction == 1 else "Legitimate"

    # Check micro transaction
    micro_flag = is_micro_transaction(amount)
    micro_msg = " (Micro Transaction)" if micro_flag else ""

    # Save to history
    history.append({
        "type": type_,
        "amount": amount,
        "result": result + micro_msg
    })

    return result + micro_msg


In [26]:
def show_history():
    if not history:
        return "No predictions made yet."
    return "\n".join([f"Type: {h['type']}, Amount: {h['amount']}, Result: {h['result']}" for h in history])


In [27]:
# Input labels: type (int), amount, balances
input_components = [
    gr.Number(label="Type (0=TRANSFER, 1=CASH_OUT, etc.)"),
    gr.Number(label="Amount"),
    gr.Number(label="Old Balance (Origin)"),
    gr.Number(label="New Balance (Origin)"),
    gr.Number(label="Old Balance (Destination)"),
    gr.Number(label="New Balance (Destination)")
]

# Output: Fraud prediction string
output_component = gr.Textbox(label="Prediction Result")


In [28]:
history_output = gr.Textbox(label="Prediction History")

# Button click function to fetch history
def fetch_history():
    return show_history()


In [31]:
import gradio as gr
import pandas as pd
from datetime import datetime
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# File paths
ACCOUNT_PATH = "/content/drive/MyDrive/fraud_detection/accounts.csv"
APPROVED_LOG = "/content/drive/MyDrive/fraud_detection/approved_transactions.csv"
FRAUD_LOG = "/content/drive/MyDrive/fraud_detection/fraud_transactions.csv"
MODEL_PATH = "/content/drive/MyDrive/fraud_detection/fraud_model.pkl"

# Load model
model = joblib.load(MODEL_PATH)

# Load accounts
def get_accounts():
    return sorted(pd.read_csv(ACCOUNT_PATH)['Account Number'].tolist())

# In-memory micro-transaction counter
micro_tx_count = {}

# Transaction Handler
def handle_transaction(sender, receiver, amount):
    global micro_tx_count
    df = pd.read_csv(ACCOUNT_PATH)

    if sender == receiver:
        return "Sender and Receiver cannot be the same.", "", "", "", ""

    try:
        amount = float(amount)
    except:
        return "Invalid amount.", "", "", "", ""

    if amount <= 0:
        return "Invalid amount.", "", "", "", ""

    try:
        sender_balance_old = df.loc[df['Account Number'] == sender, 'Balance'].values[0]
        receiver_balance_old = df.loc[df['Account Number'] == receiver, 'Balance'].values[0]
    except IndexError:
        return "Sender or Receiver account not found.", "", "", "", ""

    if sender_balance_old < amount:
        return "❌ Insufficient balance in sender's account.", f"₹{sender_balance_old:.2f}", "", "", ""

    # Micro-transaction detection
    user_key = f"{sender}->{receiver}"
    if amount < 100:
        micro_tx_count[user_key] = micro_tx_count.get(user_key, 0) + 1
    else:
        micro_tx_count[user_key] = 0

    if micro_tx_count.get(user_key, 0) >= 3:
        fraud = pd.DataFrame([{
            "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "From": sender,
            "To": receiver,
            "Attempted Amount": amount,
            "Status": "Fraud Blocked (Micro TX)",
            "Sender Balance": sender_balance_old
        }])
        fraud.to_csv(FRAUD_LOG, mode='a', index=False, header=not os.path.exists(FRAUD_LOG))
        micro_tx_count[user_key] = 0
        return (
            "🚨 Micro-transaction fraud detected! Transaction Blocked.",
            f"₹{sender_balance_old:.2f}",
            "",
            f"₹{receiver_balance_old:.2f}",
            ""
        )

    # AI Prediction
    features = pd.DataFrame([[amount, sender_balance_old, receiver_balance_old]], columns=['amount', 'sender_balance', 'receiver_balance'])
    prediction = model.predict(features)[0]
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    if prediction == 0:
        df.loc[df['Account Number'] == sender, 'Balance'] -= amount
        df.loc[df['Account Number'] == receiver, 'Balance'] += amount
        df.to_csv(ACCOUNT_PATH, index=False)

        sender_balance_new = sender_balance_old - amount
        receiver_balance_new = receiver_balance_old + amount

        approved = pd.DataFrame([{
            "Timestamp": now,
            "From": sender,
            "To": receiver,
            "Amount": amount,
            "Status": "Success",
            "Sender Balance After": sender_balance_new,
            "Receiver Balance After": receiver_balance_new
        }])
        approved.to_csv(APPROVED_LOG, mode='a', index=False, header=not os.path.exists(APPROVED_LOG))

        return (
            "✅ Transaction Successful (AI Verified)",
            f"Sender Old: ₹{sender_balance_old:.2f}",
            f"Sender New: ₹{sender_balance_new:.2f}",
            f"Receiver Old: ₹{receiver_balance_old:.2f}",
            f"Receiver New: ₹{receiver_balance_new:.2f}"
        )
    else:
        fraud = pd.DataFrame([{
            "Timestamp": now,
            "From": sender,
            "To": receiver,
            "Attempted Amount": amount,
            "Status": "Fraud Blocked (AI)",
            "Sender Balance": sender_balance_old
        }])
        fraud.to_csv(FRAUD_LOG, mode='a', index=False, header=not os.path.exists(FRAUD_LOG))

        return (
            "🚨 Fraud Detected by AI! Transaction Blocked.",
            f"₹{sender_balance_old:.2f}",
            "",
            f"₹{receiver_balance_old:.2f}",
            ""
        )

# Deposit
def deposit(account, amount):
    try:
        df = pd.read_csv(ACCOUNT_PATH)
        amount = float(amount)
        if amount <= 0:
            return "Invalid deposit amount."

        old_balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
        df.loc[df['Account Number'] == account, 'Balance'] += amount
        new_balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
        df.to_csv(ACCOUNT_PATH, index=False)

        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log = pd.DataFrame([{
            "Timestamp": now,
            "From": "CASH",
            "To": account,
            "Amount": amount,
            "Status": "Deposit",
            "Sender Balance After": "-",
            "Receiver Balance After": new_balance
        }])
        log.to_csv(APPROVED_LOG, mode='a', index=False, header=not os.path.exists(APPROVED_LOG))

        return f"✅ ₹{amount:.2f} deposited successfully. Old: ₹{old_balance:.2f}, New: ₹{new_balance:.2f}"
    except Exception as e:
        return f"❌ Deposit failed: {str(e)}"

# Withdraw
def withdraw(account, amount):
    try:
        df = pd.read_csv(ACCOUNT_PATH)
        amount = float(amount)
        current_balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
        if amount <= 0:
            return "Invalid withdraw amount."
        if amount > current_balance:
            return "Insufficient balance."

        old_balance = current_balance
        df.loc[df['Account Number'] == account, 'Balance'] -= amount
        new_balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
        df.to_csv(ACCOUNT_PATH, index=False)

        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log = pd.DataFrame([{
            "Timestamp": now,
            "From": account,
            "To": "CASH",
            "Amount": amount,
            "Status": "Withdraw",
            "Sender Balance After": new_balance,
            "Receiver Balance After": "-"
        }])
        log.to_csv(APPROVED_LOG, mode='a', index=False, header=not os.path.exists(APPROVED_LOG))

        return f"✅ ₹{amount:.2f} withdrawn successfully. Old: ₹{old_balance:.2f}, New: ₹{new_balance:.2f}"
    except Exception as e:
        return f"❌ Withdrawal failed: {str(e)}"

# Balance View
def get_balance(account):
    df = pd.read_csv(ACCOUNT_PATH)
    balance = df.loc[df['Account Number'] == account, 'Balance'].values[0]
    return f"Current Balance: ₹{balance:.2f}"

# Account Info
def get_account_info(account):
    try:
        df = pd.read_csv(ACCOUNT_PATH)
        user = df[df['Account Number'] == account].iloc[0]
        info = f"""
Name: {user['Name']}
Phone: {user['Phone']}
Address: {user['Address']}
Balance: ₹{user['Balance']:.2f}
        """
        return info.strip()
    except Exception as e:
        return f"Error: {e}"

# Fraud Table
def fraud_table():
    try:
        fraud_data = pd.read_csv(FRAUD_LOG)
        return fraud_data.to_string(index=False)
    except Exception as e:
        return f"Error loading fraud log: {e}"

# UI
with gr.Blocks() as ui:
    gr.Markdown("""
    # 🏦 AI-Based Smart Banking System with Fraud Detection
    _Secure transactions, smart banking, live AI protection_
    """)

    with gr.Tab("💸 Transaction"):
        sender = gr.Dropdown(label="Sender", choices=get_accounts())
        receiver = gr.Dropdown(label="Receiver", choices=get_accounts())
        amount = gr.Number(label="Amount")
        submit = gr.Button("Submit")
        result = gr.Textbox(label="Transaction Result")
        s_old = gr.Textbox(label="Sender Old")
        s_new = gr.Textbox(label="Sender New")
        r_old = gr.Textbox(label="Receiver Old")
        r_new = gr.Textbox(label="Receiver New")
        submit.click(handle_transaction, inputs=[sender, receiver, amount], outputs=[result, s_old, s_new, r_old, r_new])

    with gr.Tab("🏦 Deposit"):
        acc = gr.Dropdown(label="Account", choices=get_accounts())
        amt = gr.Number(label="Amount")
        out = gr.Textbox(label="Notification")
        deposit_btn = gr.Button("Deposit")
        deposit_btn.click(deposit, inputs=[acc, amt], outputs=out)

    with gr.Tab("💰 Withdraw"):
        acc2 = gr.Dropdown(label="Account", choices=get_accounts())
        amt2 = gr.Number(label="Amount")
        out2 = gr.Textbox(label="Notification")
        withdraw_btn = gr.Button("Withdraw")
        withdraw_btn.click(withdraw, inputs=[acc2, amt2], outputs=out2)

    with gr.Tab("📊 Balance"):
        acc3 = gr.Dropdown(label="Account", choices=get_accounts())
        out3 = gr.Textbox(label="Balance Info")
        bal_btn = gr.Button("Check Balance")
        bal_btn.click(get_balance, inputs=acc3, outputs=out3)

    with gr.Tab("🚨 Fraud Analysis"):
        fraud_text = gr.Textbox(label="Fraud Table", lines=10)
        view_table = gr.Button("📋 View Fraud Table")
        view_table.click(fraud_table, outputs=fraud_text)

    with gr.Tab("👤 Account Info"):
        acc_info = gr.Dropdown(label="Account", choices=get_accounts())
        show_info = gr.Button("View Info")
        output_info = gr.Textbox(label="Account Holder Info", lines=6)
        show_info.click(get_account_info, inputs=acc_info, outputs=output_info)

ui.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://559e4b112481881ea6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
